In [ ]:
import os
import shutil
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import json
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings

warnings.filterwarnings('ignore')

# TensorFlow/Keras imports (optimized for Apple Silicon)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Enable Metal Performance Shaders for Apple Silicon
if tf.config.list_physical_devices('GPU'):
    print("GPU Available - Apple Silicon Metal Performance Shaders")
else:
    print("Running on CPU")

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

GPU Available - Apple Silicon Metal Performance Shaders
TensorFlow version: 2.16.2
Keras version: 3.8.0


In [ ]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

batch_size = 16

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = "/Users/rohit/Desktop/UOA_ML_Project/CropAndWeed/crop_and_weed/extracted_patches/processed_crops_v2"

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'  # use 'categorical' if labels are one-hot
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# Normalize
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(normalize).shuffle(1000).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize).prefetch(tf.data.AUTOTUNE)

# Model
base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

from tensorflow.keras import layers, models

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_ds.cardinality().numpy(), activation='softmax')  # OR use `len(class_names)`
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model.fit(train_ds, validation_data=test_ds, epochs=50)


Found 76826 files belonging to 52 classes.
Using 61461 files for training.


2025-07-08 14:12:00.668078: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-07-08 14:12:00.668108: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-07-08 14:12:00.668110: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2025-07-08 14:12:00.668128: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-08 14:12:00.668136: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 76826 files belonging to 52 classes.
Using 15365 files for validation.
Epoch 1/50


2025-07-08 14:12:03.358862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-07-08 14:12:03.360491: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1921/1921 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.1557 - loss: 3.4212

2025-07-08 14:14:35.841797: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1921/1921 ━━━━━━━━━━━━━━━━━━━━ 202s 103ms/step - accuracy: 0.1557 - loss: 3.4210 - val_accuracy: 0.2911 - val_loss: 2.6711
Epoch 2/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 219s 112ms/step - accuracy: 0.2983 - loss: 2.5780 - val_accuracy: 0.3535 - val_loss: 2.3415
Epoch 3/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 189s 96ms/step - accuracy: 0.3822 - loss: 2.2332 - val_accuracy: 0.4112 - val_loss: 2.0811
Epoch 4/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 203s 103ms/step - accuracy: 0.4243 - loss: 2.0341 - val_accuracy: 0.4440 - val_loss: 1.9422
Epoch 5/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 201s 102ms/step - accuracy: 0.4553 - loss: 1.9079 - val_accuracy: 0.4551 - val_loss: 1.8849
Epoch 6/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 201s 103ms/step - accuracy: 0.4708 - loss: 1.8301 - val_accuracy: 0.4892 - val_loss: 1.8033
Epoch 7/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 183s 93ms/step - accuracy: 0.4871 - loss: 1.7575 - val_accuracy: 0.4809 - val_loss: 1.7681
Epoch 8/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 184s 94ms/step - accuracy: 0.4968